In [1]:
from numba.experimental import jitclass
import numba as nb
import numpy as np

spec = [
    ('J',     nb.float64),
    ('kb',    nb.float64),
    ('size',  nb.int64),
    ('temp',  nb.float64),
    ('beta',  nb.float64),
    ('system', nb.int8[:, :]),
]

@jitclass(spec)
class Ising:
    def __init__(self, size, temp, J=1.0, kb=1.0):
        self.J, self.kb = J, kb
        self.size, self.temp = size, temp
        self.beta   = 1.0 / (kb * temp)
        self.system = np.random.choice(np.int8([-1, 1]), size=(size, size))

    def LocalEnergy(self, x, y):
        L = self.size
        s = self.system[x, y]
        nn = self.system[x, (y+1)%L] + self.system[x, (y-1)%L] + \
             self.system[(x+1)%L, y] + self.system[(x-1)%L, y]
        return -self.J * s * nn

    def Glauber(self):
        L = self.size
        x = np.random.randint(L)
        y = np.random.randint(L)
        dE = -2.0 * self.LocalEnergy(x, y)
        if dE <= 0 or np.random.random() < np.exp(-self.beta * dE):
            self.system[x, y] = -self.system[x, y]

    def Simulate(self, sweeps=1):
        for _ in range(sweeps * self.size * self.size):
            self.Glauber()


In [2]:
size = 50
temps = np.linspace(1.0, 4.0, 50)
timetoequilibrium = 10000
nofsamplespertemp = 500
gapbetweensamples = 100

for temp in temps:
    ising = Ising(size, temp)
    lattices = []
    
    for i in range(timetoequilibrium):
        ising.Simulate()
    
    lattices.append(ising.system.copy())
    
    for i in range(nofsamplespertemp-1):
        for j in range(gapbetweensamples):
            ising.Simulate()
        
        lattices.append(ising.system.copy())
        
    lattices = np.array(lattices, dtype=np.int8)
    np.save(f"/Users/davidseager/Desktop/Programming/Ising/Data/Ising Systems/L{size}_T{temp:.3f}.npy", lattices)

np.save("/Users/davidseager/Desktop/Programming/Ising/Data/temperatures.npy", temps)
    